# Algorithms & Data Structures 2
Jurjen Verbruggen

# Graph generation

Below you can find several sliders to generate a nice graph. $n$ = the amount of nodes in the graph, and $p$ is the probability of generating an edge between any two nodes. Click on the Update button to generate a new graph given the two parameters, and click on Connect to join a disconnected graph together.

In [41]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from vertex_manipulations import *
import random
import networkx as nx
import matplotlib.pyplot as plt
from vcover import *
from graph import LabGraph
from graph_factory import *
from ipywidgets_debounce import *

debug_mode = False
default_n = 21
default_p = 0.8
default_k = 17

n_input = widgets.IntSlider(value = default_n, min=1, max=100, step=1, description="n")
p_input = widgets.FloatSlider(value = default_p, min=0, max=1, step=0.01, description="p")
reroll_button = widgets.Button(description="Reroll")
connect_button = widgets.Button(description="Connect")

pendant_plus_button = widgets.Button(description="p++")
pendant_minus_button = widgets.Button(description="p--")
tops_plus_button = widgets.Button(description="t++")
tops_minus_button = widgets.Button(description="t--")
highlight_kernelization_button = widgets.Button(description="Refresh")

k_vcover_input_max = n_input.value
k_vcover_input = widgets.IntSlider(value=default_k, min=1, max=k_vcover_input_max, step=1, description="k")
calculate_k_vcover_button = widgets.Button(description="Calculate")
calculated_text = widgets.Textarea(description="calculated")
calculated_one_text = widgets.Textarea()
progressbar_spec = widgets.Label(description="spec")
vcover_progressbar = widgets.IntProgress(min=0, max=1, value=0, description="progress")

calculate_vcover_mode_buttons = widgets.RadioButtons(
    options=['brute force', 'enhanced brute force'],
    value='brute force',
    description='Mode'
)
out_graph = widgets.Output()
matrix_widget = widgets.Output()

debug_view = widgets.Output(layout={'border': '1px solid black'})
logger_area = widgets.Textarea(description="log")

class State:
    displayed_graph = GraphFactory.generate_networkx_matrix(default_n, default_p)
    vcover_interrupt = 0
    ui_update_debounce = 200
    ui_update_debounce_state = ui_update_debounce

    def set_update_debounce(x: int):
        ui_update_debounce = x
        ui_update_debounce_state = x

In [42]:
generator_controls = widgets.VBox([n_input, p_input, widgets.HBox([reroll_button, connect_button])])
generator_controls

In [43]:
def draw_nx(graph):
    color_map = graph.get_color_map(k_vcover_input.value)
    nx.draw(graph, node_color=color_map, pos=nx.spring_layout(graph, seed = 1))
    nx.draw_networkx_labels(graph, pos=nx.spring_layout(graph, seed = 1))
    k_vcover_input.max = len(graph.nodes())

out_graph

Output()

In [44]:
matrix_widget

Output()

In [45]:
def update_matrix():
    with matrix_widget:
        clear_output()
        graph = State.displayed_graph
        result = graph.print_matrix()
        print(result)

def on_connect():
    with out_graph:
        graph = State.displayed_graph
        if graph is not None:
            clear_output()
            graph.connect()
            update_matrix()
            draw_nx(graph)
            plt.show()

def on_reroll():
    with out_graph:
        clear_output()
        connect_button.disabled = False
        n = n_input.value
        p = p_input.value
        graph = GraphFactory.generate_networkx_matrix(n, p)
        State.displayed_graph = graph
        calculate_k_vcover_button.disabled = False
        draw_nx(State.displayed_graph)
        plt.show()
    update_matrix()

def on_connect_button(_):
    on_connect()

def on_reroll_button(_):
    on_reroll()

reroll_button.on_click(on_reroll_button)
connect_button.on_click(on_connect_button)

def refresh_outputs():
    with out_graph:
        dg = State.displayed_graph
        if dg is not None:
            clear_output()
            color_map = dg.get_color_map(k_vcover_input.value)
            draw_nx(dg)
            plt.show()
            update_matrix()

def refresh_with_coloring():
    refresh_outputs()

def on_pendant_plus_button(_):
    State.displayed_graph.add_pendant()
    refresh_with_coloring()

def on_pendant_minus_button(_):
    State.displayed_graph.remove_pendant()
    refresh_with_coloring()

def on_tops_plus_button(_):
    k = k_vcover_input.value
    State.displayed_graph.add_tops(k)
    refresh_with_coloring()

def on_tops_minus_button(_):
    k = k_vcover_input.value
    State.displayed_graph.remove_tops(k)
    refresh_with_coloring()

def on_highlight_kernelization_button(_):
    refresh_with_coloring()

pendant_plus_button.on_click(on_pendant_plus_button)
pendant_minus_button.on_click(on_pendant_minus_button)
tops_plus_button.on_click(on_tops_plus_button)
tops_minus_button.on_click(on_tops_minus_button)
highlight_kernelization_button.on_click(on_highlight_kernelization_button)


In [46]:
def on_update_k_slider(_):
    refresh_with_coloring()

def set_progressbar(val):
    max = vcover_progressbar.max
    quo = val/max*100
    progressbar_spec.value = "%.2f" % quo + "% (" + str(val) + "/" + str(max) + ")"
    if val > max: return
    vcover_progressbar.value = val

def complete_progressbar():
    set_progressbar(vcover_progressbar.max)
    State.ui_update_debounce_state = State.ui_update_debounce

def update_progressbar(x: int):
    State.ui_update_debounce_state -= 1
    if State.ui_update_debounce_state > 0:
        return
    State.ui_update_debounce_state = State.ui_update_debounce

    val = vcover_progressbar.value
    set_progressbar(x+val)
    
def update_calculated_text(x: str):
    calculated_text.value = x

def on_calculate_k_vcover_button(_):
    calculate_k_vcover_button.disabled = True
    if State.displayed_graph == None:
        return

    g = State.displayed_graph
    nodes = list(g.nodes())
    edges = list(g.edges())
    k = k_vcover_input.value
    n = len(nodes)
    enhanced_mode = calculate_vcover_mode_buttons.value == "enhanced brute force"

    vcover_progressbar.value = 0
    vcovers = None

    pbar_incr = lambda x : update_progressbar(x)
    pbar_desc = lambda x : update_calculated_text(x)

    if enhanced_mode:
        complx = 2**k+n
        State.set_update_debounce(int(complx/100))
        vcover_progressbar.max = complx

        vcovers = find_vcovers_enhanced(nodes, edges, n, k, pbar_incr, pbar_desc)
    else:
        complx = 2**k # Not n because brute force stops if len(VC) > k
        State.set_update_debounce(int(complx/100))
        vcover_progressbar.max = complx
        vcovers = find_vcovers_brute(nodes, edges, n, k, pbar_incr, pbar_desc)
    
    complete_progressbar()
    if vcovers == None or len(vcovers) == 0:
        update_calculated_text("No v-cover possible for k=" + str(k))
    else:
        update_calculated_text("Possible v-covers: \n" + str(vcovers))
        vcover = vcovers[0]
        calculated_one_text.value = str(vcover)
    
    calculate_k_vcover_button.disabled = False

calculate_k_vcover_button.on_click(on_calculate_k_vcover_button)

box_week2 = widgets.VBox([
    widgets.HBox([k_vcover_input, highlight_kernelization_button]), 
    widgets.HBox([calculate_vcover_mode_buttons, calculate_k_vcover_button]), 
    widgets.HBox([calculated_text, calculated_one_text]), 
    vcover_progressbar, 
    progressbar_spec,
    widgets.HBox([pendant_plus_button, pendant_minus_button]),
    widgets.HBox([tops_plus_button, tops_minus_button])
])
cpanel = widgets.HBox([
    box_week2,
    widgets.VBox([
        logger_area
    ])
])

if debug_mode:
    on_calculate_k_vcover_button(1)
cpanel